<a href="https://colab.research.google.com/github/zhouchun0105/credit-card-churners/blob/main/SUBMIT_creditcard_churners_team1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task:
1. Finish this documentation by Wednesday
2. One person at least choose to answer 2-3 questions regarding to the dataset. Specify the question in the "Following are a few questions that we aim to answer through our analysis:" session
3. Meeting on Tuesday once and Wednesday again to confirm final submission

Note:
- Chun works for Overall Documentation and ML (optional)
- Ana works for Q2 and Q12 and ML (optional)
- For Ann Lu, Phyllis, Zijing and Yuhan, try to finish 2-4 questions, depend on the complexity of your question
- Chun will finish the layout/formatting of documentation on Monday evening so everyone can fill in your part

# **Credit Card Customer Churn Analysis and Predictions**

Team 1, Afternoon Session

Ana Maharjan, Ann Lu, Chun Zhou, Phyllis Cao, Yuhan Wang, Zijing Wang

2021/08/26





# **Introduction**

Something here

## **Following are a few questions that we aim to answer through our analysis**


1.	What is the churn rate of our customer?
2.	What is our user portrait like in terms of age, gender, educational level and income?
3.	What age group of customers has a higher tendency to churn?
4.	Which education category of customers has a higher tendency to churn?
5.	Which income category of customers has a higher tendency to churn?
6.	Which marriage type of customers has a higher tendency to churn?
7.	How does dependent count influence churn rate?
8.	Which card type of customers has a higher tendency to churn?
9.	How does credit limit affect churn rate?
10.	How does the number of contacts by sales representative in the past 12 month affect churn rate?
11.	How does time length as being our customer affect churn rate?
12.	How does the number of other existing service affect churn rate?
13.	How does transaction amount affect churn rate?
14.	How does revolving balance affect churn rate?




---


Explanation (to be deleted): 
Q11 refer to month on book column
Q12 refer to total relationship count column


---






## **Description of Data**

Something here

### **Question 1: What is the churn rate of our customer?**

### **Question 2: What is our user portrait like in terms of age, gender, educational level and income?**

### **Question 3: What age group of customers has a higher tendency to churn?**

### **Question 4: Which education category of customers has a higher tendency to churn?**

### **Question 5: Which income category of customers has a higher tendency to churn?**

### **Question 6: Which marriage type of customers has a higher tendency to churn?**

### **Question 7: How does dependent count influence churn rate?**

### **Question 8: Which card type of customers has a higher tendency to churn?**

### **Question 9: How does credit limit affect churn rate?**

### **Question 10: How does the number of contacts by sales representative in the past 12 month affect churn rate?**

### **Question 11: How does time length as being our customer affect churn rate?**

### **Question 12: How does the number of other existing service affect churn rate?**

### **Question 13: How does transaction amount affect churn rate?**

### **Question 14: How does revolving balance affect churn rate?**